<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/10-Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow.

Let's get our vectorDB from before.

In [ ]:
!pip install openai==0.28
!pip install langchain langchain-community

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

### OpenAI 설치  Ver 확인

In [3]:
!pip show openai|grep Version #OpenAI 설치  Ver 확인

Version: 0.28.0


### Similarity Search

In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 13.5 MB/s et

In [5]:
!pip install tiktoken

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [7]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

<ipython-input-7-2897838732>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
<ipython-input-7-2897838732>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [8]:
print(vectordb._collection.count())

0


In [9]:
texts = [
    """광대버섯(Amanita phalloides)은 크고 인상적인 후성(위) 자실체(담자과체)를 가지고 있습니다.""",
    """큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.""",
    """A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.""",
]

In [10]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [11]:
question = "큰 자실체를 가진 순백색 버섯에 대해 알려주세요"

In [12]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.'),
 Document(metadata={}, page_content='A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.')]

In [13]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={}, page_content='큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.'),
 Document(metadata={}, page_content='A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.')]